导入数据集

In [1]:
import warnings
warnings.filterwarnings("ignore")
import modin.pandas as pd
data=pd.read_csv("creditcard.csv")

检查数据分布

In [2]:
from termcolor import colored as cl
Total_transactions = len(data)
normal = len(data[data.Class == 0])
fraudulent = len(data[data.Class == 1])
fraud_percentage = round(fraudulent/normal*100, 2)
print(cl('Total number of Trnsactions are {}'.format(Total_transactions), attrs = ['bold']))
print(cl('Number of Normal Transactions are {}'.format(normal), attrs = ['bold']))
print(cl('Number of fraudulent Transactions are {}'.format(fraudulent), attrs = ['bold']))
print(cl('Percentage of fraud Transactions is {}'.format(fraud_percentage), attrs = ['bold']))

Total number of Trnsactions are 284807
Number of Normal Transactions are 284315
Number of fraudulent Transactions are 492
Percentage of fraud Transactions is 0.17


检查空值，得到没有空值

In [3]:
data.info()

<class 'modin.pandas.dataframe.DataFrame'>
RangeIndex: 284807 entries, 0 to 284806
Data columns (total 31 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Time    284807 non-null  float64
 1   V1      284807 non-null  float64
 2   V2      284807 non-null  float64
 3   V3      284807 non-null  float64
 4   V4      284807 non-null  float64
 5   V5      284807 non-null  float64
 6   V6      284807 non-null  float64
 7   V7      284807 non-null  float64
 8   V8      284807 non-null  float64
 9   V9      284807 non-null  float64
 10  V10     284807 non-null  float64
 11  V11     284807 non-null  float64
 12  V12     284807 non-null  float64
 13  V13     284807 non-null  float64
 14  V14     284807 non-null  float64
 15  V15     284807 non-null  float64
 16  V16     284807 non-null  float64
 17  V17     284807 non-null  float64
 18  V18     284807 non-null  float64
 19  V19     284807 non-null  float64
 20  V20     284807 non-null  float64
 21  V21  

检查Amount的最小值和最大值

In [4]:
min(data.Amount),max(data.Amount)

(0.0, 25691.16)

整理amount，将数据转换为均值为0，标准差为1的分布

In [5]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
amount = data['Amount'].values
data['Amount'] = sc.fit_transform(amount.reshape(-1, 1))

检查重复数据

In [6]:
data.shape

(284807, 31)

去重

In [7]:
data.drop_duplicates(inplace=True)

In [8]:
data.shape

(283726, 31)

拆分训练集与测试集，随机欠采样

In [9]:
from sklearnex import patch_sklearn
patch_sklearn()
from sklearn.model_selection import train_test_split
X = data.drop('Class', axis = 1).values
y = data['Class'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# # 随机欠采样
# from imblearn.under_sampling import RandomUnderSampler
# rus = RandomUnderSampler(random_state=42)
# X_train_resampled, y_train_resampled = rus.fit_resample(X_train, y_train)
# 创建TomekLinks对象
from imblearn.under_sampling import TomekLinks
tl = TomekLinks(sampling_strategy='auto')
# 应用Tomek Links
X_train, y_train = tl.fit_resample(X_train, y_train)

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


决策树

In [10]:
from sklearn.tree import DecisionTreeClassifier
DT = DecisionTreeClassifier(max_depth = 4, criterion = 'entropy')
DT.fit(X_train, y_train)
dt_yhat = DT.predict(X_test)

F1分数

In [11]:
from sklearn.metrics import f1_score
print('F1 score of the Decision Tree model is {}'.format(f1_score(y_test, dt_yhat)))

F1 score of the Decision Tree model is 0.7976878612716763


随机森林

In [12]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(max_depth = 4)
rf.fit(X_train, y_train)
rf_yhat = rf.predict(X_test)
print('F1 score of the Random Forest model is {}'.format(f1_score(y_test, rf_yhat)))

F1 score of the Random Forest model is 0.6666666666666667


AdaBoost

In [13]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
base_classifier = DecisionTreeClassifier(max_depth=2)  
adaboost_model = AdaBoostClassifier(base_classifier, n_estimators=10)  
adaboost_model.fit(X_train, y_train)
ada_yhat = adaboost_model.predict(X_test)
print('F1 score of the AdaBoost model is {}'.format(f1_score(y_test, ada_yhat)))

F1 score of the AdaBoost model is 0.7468354430379747
